# MNIST MLP - Dropout + Sigmoid (default)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn # 인공 신경망 모델을 설계할 때 필요한 함수를 모아 놓은 모듈
import torch.nn.functional as F # torch.nn 모듈 중에서도 자주 이용되는 함수를 'F'로 지정
import torch.nn.init as init # Weight, Bias 등 딥러닝 모델에서 초깃값으로 설정되는 요소에 대한 모듈
from torchvision import transforms, datasets # 컴퓨터 비전 연구 분야에서 자주 이용하는 torchvision 모듈

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.7.1  Device: cpu


In [3]:
# 하이퍼파라미터 지정
BATCH_SIZE = 32  # 미니배치 1개 단위에 대해 데이터가 32개로 구성되어 있는 것을 의미
EPOCHS = 10      # 전체 데이터 셋을 10번 반복해 학습한다는 것을 의미 (즉, 전체 데이터를 이용해 학습을 진행한 횟수)

# MNIST 데이터 셋 다운로드
train_dataset = datasets.MNIST(root = '../data/MNIST',  # 데이터 셋이 저장될 장소를 지정
                               train = True,            # 학습용 데이터 셋으로 지정 
                               download = True,         # 인터넷 상에서 다운로드해서 이용할 것인지 여부
                               transform = transforms.ToTensor()) # 데이터 셋을 tensor 형태로 변경
test_dataset = datasets.MNIST(root = '../data/MNIST',
                              train = False,
                              transform = transforms.ToTensor())
# 다운로드한 MNIST 데이터 셋을 미니배치 단위로 분리
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE, # 미니배치 1개 단위를 구성하는 데이터의 개수
                                           shuffle = True) # 데이터의 순서를 섞고자 할 때 이용(즉, 잘못된 방향으로 학습하는 것을 방지)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [4]:
# 데이터 확인하기 (1)
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [5]:
### 아래의 코드를 실행하면 Kernel이 꺼진다..왜 이러지?
# pltsize = 1
# plt.figure(figsize = (10 * pltsize, pltsize))
# for i in range(10):
#     plt.subplot(1, 10, i + 1)
#     plt.axis('off')
#     plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
#     plt.title('Class: ' + str(y_train[i].item()))

<참고>
- ```training = self.training```
    - Dropout은 학습 과정 속에서 랜덤으로 노드를 선택해 가중값이 업데이트되지 않도록 조정하지만, 평가 과정 속에서는 모든 노드를 이용하여 Output을 계산하기 때문에 학습 상태와 검증 상태에서 다르게 적용되어야 한다.
    - 이를 반영하기 위해 다음과 같이 파라미터를 다르게 설정해줘야 한다.
        1. 파라미터 값을 ```model.train()```으로 명시할 때는 ```self.training = True```로 적용
        2. 파라미터 값을 ```model.eval()```으로 명시할 때는 ```self.training = False```로 적용

In [6]:
# Multi Layer Perceptron (MLP) 모델 설계하기
# PyTorch 모듈 내에 딥러닝 모델 관련 기본 함수를 포함하고 있는 nn.Module 클래스를 상속받는 Net 클래스를 정의
class Net(nn.Module):
    def __init__(self): # Net 클래스의 인스턴스를 생성했을 때 지니게 되는 성질을 정의
        super(Net, self).__init__()        # nn.Module 내에 있는 메서드를 상속받아 사용
        self.fc1 = nn.Linear(28 * 28, 512) # 첫 번째 Fully Connected Layer 정의
        self.fc2 = nn.Linear(512, 256)     # 두 번째 Fully Connected Layer 정의
        self.fc3 = nn.Linear(256, 10)      # 세 번째 Fully Connected Layer 정의
        self.dropout_prob = 0.5            # 50%의 노드에 대해 가중값을 계산하지 않도록 설정
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # view 메서드를 통해 2차원 데이터를 784 크기의 1차원 데이터로 변환
        
        x = self.fc1(x)          # __init__()을 통해 정의한 첫 번째 Fully Connected Layer에 1차원으로 펼친 이미지 데이터를 통과
        x = F.sigmoid(x)         # 비선형 함수인 sigmoid()를 이용하여 두 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc2(x)          # __init__()을 통해 정의한 두 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = F.sigmoid(x)         # 비선형 함수인 sigmoid()를 이용하여 세 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc3(x)                # __init__()을 통해 정의한 세 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = F.log_softmax(x, dim = 1)  # log_softmax()를 이용하여 최종 Output을 계산
        # <참고>
        # softmax가 아닌 log_softmax를 사용하는 이유는 MLP 모델이 역전파 알고리즘을 이용해 학습을 진행할 때,
        # Loss 값에 대한 Gradient 값을 좀 더 원활하게 계산할 수 있기 때문!
        return x

In [7]:
# Optimizer, Objective Function 설정하기
model = Net().to(DEVICE) # 'DEVICE' 장비를 이용하여 MLP 모델을 완성하기 위해, 앞서 정의한 MLP 모델을 기존에 선정한 'DEVICE'에 할당
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # momentum은 Optimizer의 관성을 의미
criterion = nn.CrossEntropyLoss() # MLP 모델의 Output 값과 원-핫 인코딩 값인 Label 값의 Loss를 CrossEntropy를 이용하여 계산

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [8]:
# MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 (MLP 모델을 학습)
def train(model, train_loader, optimizer, log_interval):
    model.train() # 기존에 정의한 MLP 모델을 '학습' 상태로 지정
    # train_loader 내에 미니배치 단위로 저장된 데이터를 순서대로 이용하여 MLP 모델을 학습
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE) # 미니배치 내에 있는 image 데이터를 이용하여 MLP 모델을 학습시키기 위해 기존에 정의한 장비에 할당
        label = label.to(DEVICE) # 미니배치 내에 있는 image 데이터와 매칭된 label 데이터도 기존에 정의한 장비에 할당
        
        optimizer.zero_grad()    # opimizer의 Gradient를 초기화
        
        output = model(image)    # 장비에 할당한 이미지 데이터를 MLP 모델의 Input으로 이용
        loss = criterion(output, label)
        
        loss.backward()   # Loss 값을 계산한 결과를 바탕으로 역전파를 통해 계산된 Gradient 값을 각 파라미터에 할당
        optimizer.step()  # 각 파라미터에 할당된 Gradient 값을 이용하여 파라미터 값을 업데이트
        
        # 아래의 코드는 위 함수가 실행되는 과정을 모니터링하기 위한 코드
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
# 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 (학습의 진행 과정을 모니터링)
def evaluate(model, test_loader):
    model.eval()   # 학습 과정 또는 학습이 완료된 MLP 모델을 학습 상태가 아닌 '평가' 상태로 지정
    test_loss = 0  # 기존에 정의한 test_loader 내의 데이터를 이용하여 Loss 값을 계산하기 위해 '0'으로 임시 설정
    correct = 0    # 학습 과정 또는 학습이 완료된 MLP 모델이 올바른 Class로 분류한 경우를 count 하기 위해 '0'으로 임시 설정
    
    # MLP 모델을 평가하는 단계에서는 Gradient를 통해 파라미터 값이 업데이트 되는 현상을 방지하기 위해,
    # torch.no_grad() 메서드를 통해 Gradient의 흐름을 억제
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)  # 미니배치 내에 있는 image 데이터를 이용하여 MLP 모델을 검증하기 위해 기존에 정의한 장비에 할당
            label = label.to(DEVICE)  # 미니배치 내에 있는 image 데이터와 매칭된 label 데이터도 기존에 정의한 장비에 할당
            
            output = model(image)     # 장비에 할당한 image 데이터를 MLP 모델의 Input으로 사용
            test_loss += criterion(output, label).item()   # test_loss 값 업데이트
            
            prediction = output.max(1, keepdim = True)[1]  # 가장 큰 벡터 값의 위치에 대응하는 클래스로 예측했다고 판단
            correct += prediction.eq(label.view_as(prediction)).sum().item() # 최종 예측이 올바른지 count
            
    test_loss /= len(test_loader.dataset) # 현재까지 계산된 test_loss 값을 미니배치 개수만큼 나눠서 평균 Loss 값을 계산
    test_accuracy = 100. * correct / len(test_loader.dataset) # 정확도 계산
    return test_loss, test_accuracy

In [10]:
# MLP 모델 학습을 실행하며 Train/Test set의 Loss 및 Test set Accuracy 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) # MLP 모델 학습 with SGD Optimizer
    test_loss, test_accuracy = evaluate(model, test_loader)   # 각 epoch별로 출력되는 Loss 값과 accuracy 값을 계산
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n]'.format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.467475


C:\Users\Brian\anaconda3\envs\DL_torch_cpu\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 2.271861
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 2.291526
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 2.400225
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 2.359376
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 2.319067
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 2.339031
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 2.341093
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 2.274296
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 2.297570

[EPOCH: 1], 	Test Loss: 0.0714, 	Test Accuracy: 10.79 %
]
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 2.315763
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 2.276724
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 2.307859
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 2.271694
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 2.276572
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 2.239483
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 2.234201
Train Epoch: 2 [44800/60000 (75%)]	Train Loss: 2.106

# MNIST MLP - Dropout + ReLU
- ReLU 함수는 0 미만인 값은 0으로 계산하고, 양수 값은 그대로 반영하는 비선형 함수이다.
- Dropout은 보통 비선형 함수인 ReLu 함수와 잘 어울린다.
    - Sigmoid 함수는 0에서 멀어질수록 Gradient 값이 0에 가까워지므로, Back Propagation이 효과적으로 이용되기 어렵다.
    - 반면에, ReLU 함수는 **Gradient Vanishing 문제를 어느 정도 해결**해준다.

<참고>
- "Multi Layer Perceptron (MLP) 모델 설계" 부분을 제외한 나머지 과정은 앞과 동일하므로 생략하겠다.

In [11]:
# Multi Layer Perceptron (MLP) 모델 설계하기
# PyTorch 모듈 내에 딥러닝 모델 관련 기본 함수를 포함하고 있는 nn.Module 클래스를 상속받는 Net 클래스를 정의
class Net(nn.Module):
    def __init__(self): # Net 클래스의 인스턴스를 생성했을 때 지니게 되는 성질을 정의
        super(Net, self).__init__()        # nn.Module 내에 있는 메서드를 상속받아 사용
        self.fc1 = nn.Linear(28 * 28, 512) # 첫 번째 Fully Connected Layer 정의
        self.fc2 = nn.Linear(512, 256)     # 두 번째 Fully Connected Layer 정의
        self.fc3 = nn.Linear(256, 10)      # 세 번째 Fully Connected Layer 정의
        self.dropout_prob = 0.5            # 50%의 노드에 대해 가중값을 계산하지 않도록 설정
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # view 메서드를 통해 2차원 데이터를 784 크기의 1차원 데이터로 변환
        
        x = self.fc1(x)          # __init__()을 통해 정의한 첫 번째 Fully Connected Layer에 1차원으로 펼친 이미지 데이터를 통과
        x = F.relu(x)            # 비선형 함수인 relu()를 이용하여 두 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc2(x)          # __init__()을 통해 정의한 두 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = F.relu(x)            # 비선형 함수인 relu()를 이용하여 세 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc3(x)                # __init__()을 통해 정의한 세 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = F.log_softmax(x, dim = 1)  # log_softmax()를 이용하여 최종 Output을 계산
        # <참고>
        # softmax가 아닌 log_softmax를 사용하는 이유는 MLP 모델이 역전파 알고리즘을 이용해 학습을 진행할 때,
        # Loss 값에 대한 Gradient 값을 좀 더 원활하게 계산할 수 있기 때문!
        return x

In [12]:
# Optimizer, Objective Function 설정하기
model = Net().to(DEVICE) # 'DEVICE' 장비를 이용하여 MLP 모델을 완성하기 위해, 앞서 정의한 MLP 모델을 기존에 선정한 'DEVICE'에 할당
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # momentum은 Optimizer의 관성을 의미
criterion = nn.CrossEntropyLoss() # MLP 모델의 Output 값과 원-핫 인코딩 값인 Label 값의 Loss를 CrossEntropy를 이용하여 계산

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [13]:
# MLP 모델 학습을 실행하며 Train/Test set의 Loss 및 Test set Accuracy 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) # MLP 모델 학습 with SGD Optimizer
    test_loss, test_accuracy = evaluate(model, test_loader)   # 각 epoch별로 출력되는 Loss 값과 accuracy 값을 계산
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n]'.format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.318754
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 2.104750
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 1.228628
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.843933
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.691317
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.748156
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.724201
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.505370
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.541805
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.270850

[EPOCH: 1], 	Test Loss: 0.0100, 	Test Accuracy: 90.95 %
]
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.219867
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.301647
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.247123
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.344572
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.370230
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.361282
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.136813
T

- 결과를 보면, sigmoid() 함수를 사용했을 때보다 ReLU() 함수를 사용했을 때 성능이 좋아진 것을 확인할 수 있다.

# MNIST MLP - Dropout + ReLU + Batch Normalization
- 신경망에는 과적합과 Gradient Vanishing 외에도 **Internal Covariance shift**라는 현상이 발생한다.
- Internal Covariance shift란, 각 Layer마다 Input 분포가 달라짐에 따라 학습 속도가 느려지는 현상을 말한다.
- 이를 방지하기 위해, Batch Normalization을 사용한다.
    - Layer의 Input 분포를 정규화해서 학습 속도를 빠르게 하겠다는 것! (즉, 정규화를 통해 비선형 활성 함수의 의미를 살리는 개념)
    - Batch Normalization을 사용하면 학습 속도를 향상시켜주고, Gradient Vanishing 문제도 완화해준다.

<참고>
- "Multi Layer Perceptron (MLP) 모델 설계" 부분을 제외한 나머지 과정은 앞과 동일하므로 생략하겠다.
- 또한 Batch Normalization은 1차원, 2차원, 3차원 등 다양한 차원에 따라 적용되는 함수명이 다르기 때문에 유의해서 사용해야 한다.
    - MLP 내 각 Layer에서 데이터는 1차원 크기의 벡터 값을 계산하기 때문에 ```nn.BatchNorm1d()```를 이용한다.
- ```nn.BatchNorm()``` 함수를 이용해 적용하는 부분은 연구자들의 선호에 따라 다음과 같이 2가지 경우로 나뉜다.
    1. Activation Function 이전에 적용 (아래의 예제에서는 이 방법을 사용하였음)
    2. Activation Function 이후에 적용

In [14]:
# Multi Layer Perceptron (MLP) 모델 설계하기
# PyTorch 모듈 내에 딥러닝 모델 관련 기본 함수를 포함하고 있는 nn.Module 클래스를 상속받는 Net 클래스를 정의
class Net(nn.Module):
    def __init__(self): # Net 클래스의 인스턴스를 생성했을 때 지니게 되는 성질을 정의
        super(Net, self).__init__()        # nn.Module 내에 있는 메서드를 상속받아 사용
        self.fc1 = nn.Linear(28 * 28, 512) # 첫 번째 Fully Connected Layer 정의
        self.fc2 = nn.Linear(512, 256)     # 두 번째 Fully Connected Layer 정의
        self.fc3 = nn.Linear(256, 10)      # 세 번째 Fully Connected Layer 정의
        self.dropout_prob = 0.5            # 50%의 노드에 대해 가중값을 계산하지 않도록 설정
        self.batch_norm1 = nn.BatchNorm1d(512) # 첫 번째 Fully Connected Layer의 Output이 512 크기의 벡터값 
        self.batch_norm2 = nn.BatchNorm1d(256) # 두 번째 Fully Connected Layer의 Output이 256 크기의 벡터값 
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # view 메서드를 통해 2차원 데이터를 784 크기의 1차원 데이터로 변환
        
        x = self.fc1(x)          # __init__()을 통해 정의한 첫 번째 Fully Connected Layer에 1차원으로 펼친 이미지 데이터를 통과
        x = self.batch_norm1(x)  # Activation Function 이전에 Batch Normalization을 적용
        x = F.relu(x)            # 비선형 함수인 relu()를 이용하여 두 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc2(x)          # __init__()을 통해 정의한 두 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = self.batch_norm2(x)  # Activation Function 이전에 Batch Normalization을 적용
        x = F.relu(x)            # 비선형 함수인 relu()를 이용하여 세 번째 Fully Connected Layer의 Input으로 계산
        # sigmoid() 함수의 결과값에 Dropout을 적용
        x = F.dropout(x, 
                      training = self.training,  # 학습 상태일 떄와 검증 상태에 따라 다르게 적용시키기 위한 파라미터
                      p = self.dropout_prob)     # 몇 %의 노드에 대해 계산하지 않을 것인지를 설정
        
        x = self.fc3(x)                # __init__()을 통해 정의한 세 번째 Fully Connected Layer에 앞서 계산된 x를 통과
        x = F.log_softmax(x, dim = 1)  # log_softmax()를 이용하여 최종 Output을 계산
        # <참고>
        # softmax가 아닌 log_softmax를 사용하는 이유는 MLP 모델이 역전파 알고리즘을 이용해 학습을 진행할 때,
        # Loss 값에 대한 Gradient 값을 좀 더 원활하게 계산할 수 있기 때문!
        return x

In [15]:
# Optimizer, Objective Function 설정하기
model = Net().to(DEVICE) # 'DEVICE' 장비를 이용하여 MLP 모델을 완성하기 위해, 앞서 정의한 MLP 모델을 기존에 선정한 'DEVICE'에 할당
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # momentum은 Optimizer의 관성을 의미
criterion = nn.CrossEntropyLoss() # MLP 모델의 Output 값과 원-핫 인코딩 값인 Label 값의 Loss를 CrossEntropy를 이용하여 계산

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [16]:
# MLP 모델 학습을 실행하며 Train/Test set의 Loss 및 Test set Accuracy 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) # MLP 모델 학습 with SGD Optimizer
    test_loss, test_accuracy = evaluate(model, test_loader)   # 각 epoch별로 출력되는 Loss 값과 accuracy 값을 계산
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n]'.format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.631368
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.587032
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.428954
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.562645
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.339285
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.324099
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.242714
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.401987
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.159260
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.243362

[EPOCH: 1], 	Test Loss: 0.0049, 	Test Accuracy: 95.26 %
]
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.192757
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.312221
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.205559
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.181145
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.363126
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.293332
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.192494
T

# MNIST MLP - Dropout + ReLU + Batch Normalization + He
- 신경망은 처음에 Weight를 랜덤하게 초기화하고 Loss가 최소화되는 부분을 찾아간다.
- 따라서 Weight의 초깃값을 어떻게 설정하느냐에 따라 학습 속도가 달라질 수 있다.
    - 대표적인 초깃값으로는 LeCun, Xavier, He가 있다.
        - ReLU 함수를 사용할 때 비효율적인 Xavier 초깃값을 보완한 것이 He 초깃값이다.

<참고>
- "Optimizer, Objective Function 설정" 부분을 제외한 나머지 과정은 앞과 동일하므로 생략하겠다.
- PyTorch 내의 ```nn.Linear```는 기본값으로 **균등 분포(Uniform Distribution)**에서 샘플링을 통해 파라미터를 초기화한다.

<참고>
- ```torch.nn.init``` 모듈 참조 사이트
    - https://pytorch.org/docs/stable/nn.init.html

In [17]:
# MLP 모델 내의 Weight를 초기화할 부분을 설정하기 위한 함수
def weight_init(m):
    if isinstance(m, nn.Linear):              # MLP 모델을 구성하고 있는 파라미터 중, nn.Linear에 해당하는 파라미터 값에 대해서만 지정
        init.kaiming_uniform_(m.weight.data)  # nn.Linear에 해당하는 파라미터 값에 대해 He 초깃값을 이용해 파라미터 값을 초기화

# Optimizer, Objective Function 설정하기
model = Net().to(DEVICE) # 'DEVICE' 장비를 이용하여 MLP 모델을 완성하기 위해, 앞서 정의한 MLP 모델을 기존에 선정한 'DEVICE'에 할당
model.apply(weight_init) # weight_init 함수를 Net 클래스의 인스턴스인 model에 적용
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) # momentum은 Optimizer의 관성을 의미
criterion = nn.CrossEntropyLoss() # MLP 모델의 Output 값과 원-핫 인코딩 값인 Label 값의 Loss를 CrossEntropy를 이용하여 계산

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [18]:
# MLP 모델 학습을 실행하며 Train/Test set의 Loss 및 Test set Accuracy 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) # MLP 모델 학습 with SGD Optimizer
    test_loss, test_accuracy = evaluate(model, test_loader)   # 각 epoch별로 출력되는 Loss 값과 accuracy 값을 계산
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n]'.format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 3.170011
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.890471
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.518925
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.906364
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.617774
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.843424
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.843766
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.104850
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.190667
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.247100

[EPOCH: 1], 	Test Loss: 0.0068, 	Test Accuracy: 93.50 %
]
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.671495
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.474878
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.301049
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.161757
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.234469
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.207072
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.319029
T

# MNIST MLP - Dropout + ReLU + Batch Normalization + He + Adam
- 많이 사용되는 Stochastic Gradient Descent(SGD) 외에도, 다음과 같이 다양한 Optimizer가 존재한다.
    1. Momentum
        - SGD보다 최적의 장소로 더 빠르게 수렴(즉, 보폭을 크게 하는 개념)한다.
        - 최적해가 아닌 지역해를 지나칠 수 있다는 장점이 있다.
    2. Nesterov Accelerated Gradient(NAG)
        - Momentum을 변형한 방법이다.
    3. Adaptive Gradient(Adagrad)
        - Adagrad의 개념은 "가보지 않은 곳은 많이 움직이고, 가본 곳은 조금씩 움직이자"이다.
    4. RMSProp
        - Adagrad의 단점을 보완한 방법이다.
            - Adagrad는 학습이 오래 진행될수록 부분이 계속 증가해 step size가 작아진다는 문제가 있다.
    5. Adaptive Delta(Adadelta)
        - Adagrad의 단점을 보완한 방법이다.
        - Gradient의 양이 너무 적어지면 움직임이 멈출 수 있는데, 이를 방지하기 위한 방법이다.
    6. **Adaptive Moment Estimation(Adam)**
        - 딥러닝 모델을 디자인할 때, **기본적으로 가장 많이 사용하는 Optimizer**이다.
        - RMSEProp과 Momentum 방식의 특징을 결합한 방법이다.
    7. Rectified Adam Optimizer(RAdam)
        - 대부분의 Optimizer는 학습 초기에 Bad Local Optimum에 수렴해 버릴 수 있는 단점이 있다.
            - 즉, 학습 초기에 Gradient가 매우 작아 져서 학습이 더 이상 일어나지 않는 현상이 발생
        - 이러한 Adaptive Learning Rate Term의 분산을 교정(Recify)하는 Optimizer이다.
            - Learning Rate를 어떻게 조절하든 성능이 비슷하다.
            - 즉, Learning Rate에 민감하지 않다.

In [19]:
# MLP 모델 내의 Weight를 초기화할 부분을 설정하기 위한 함수
def weight_init(m):
    if isinstance(m, nn.Linear):              # MLP 모델을 구성하고 있는 파라미터 중, nn.Linear에 해당하는 파라미터 값에 대해서만 지정
        init.kaiming_uniform_(m.weight.data)  # nn.Linear에 해당하는 파라미터 값에 대해 He 초깃값을 이용해 파라미터 값을 초기화

# Optimizer, Objective Function 설정하기
model = Net().to(DEVICE) # 'DEVICE' 장비를 이용하여 MLP 모델을 완성하기 위해, 앞서 정의한 MLP 모델을 기존에 선정한 'DEVICE'에 할당
model.apply(weight_init) # weight_init 함수를 Net 클래스의 인스턴스인 model에 적용
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, momentum = 0.5) # momentum은 Optimizer의 관성을 의미
criterion = nn.CrossEntropyLoss() # MLP 모델의 Output 값과 원-핫 인코딩 값인 Label 값의 Loss를 CrossEntropy를 이용하여 계산

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [20]:
# MLP 모델 학습을 실행하며 Train/Test set의 Loss 및 Test set Accuracy 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200) # MLP 모델 학습 with SGD Optimizer
    test_loss, test_accuracy = evaluate(model, test_loader)   # 각 epoch별로 출력되는 Loss 값과 accuracy 값을 계산
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %\n]'.format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 2.930486
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.225214
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.913058
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.329149
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.204424
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.203714
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.510089
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.242505
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.241195
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.109743

[EPOCH: 1], 	Test Loss: 0.0041, 	Test Accuracy: 95.66 %
]
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.347754
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.185216
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.187265
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.332085
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.342244
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.238569
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.166769
T

# FashionMNIST - AutoEncoder
1. AutoEncoder(AE)
    - 대표적인 **비지도학습 신경망 모델**이다.
    - AutoEncoder를 활용하면 Input Data를 Latent Space에 압축시켜, 이 값을 새로운 Feature로 사용할 수 있다.
        - 즉, **Feature Extraction의 일종**으로 새로운 Feature를 사용했을 때 기존의 Feature를 사용할 때보다 성능이 좋고, 차원을 줄일 수 있다는 장점이 있다.
    - AutoEncoder의 학습 과정은 **데이터를 원래의 데이터로 잘 복원하도록 학습시키는 것**으로 이해할 수 있다.
    
2. Stacked AutoEncoder(SAE)
    - 말 그대로 **AutoEncoder를 쌓아올린 모델**이다.
        - 즉, AutoEncoder의 새로운 Feature가 Feature로서의 의미가 있다면, 이를 쌓아 올려서 학습하면 더 좋은 학습 모델을 만들 수 있을 것이라는 생각으로부터 만들어진 모델이다.
            - 다시 말해, **"좋은 Feature를 지니고 있는 Hidden Layer를 쌓아 네트워크를 학습시키면 더 좋은 모델을 만들 수 있을 것이다"**라는 개념이다.
        - 학습 과정은 다음과 같다.
            1. Input Data로 AutoEncoder1을 학습
            2. 1번에서 학습된 모형의 Hidden Layer를 Input으로 해서 AutoEncoder2를 학습
            3. 2번 과정을 원하는 만큼 반복
            4. 1 ~ 3번 과정에서 학습된 Hidden Layer를 쌓아 올림
            5. 마지막 Layer에 Softmax와 같은 분류 기능이 있는 Output Layer를 추가
            6. Fine-tuning으로 전체 다충 신경망을 재학습
                - **Pre-trained Model**: 미리 학습시킨 모델
                - **Fine-tuning**: 따로 학습시킨 모델을 재학습시키는 개념(즉, Pre-trained Model을 재학습시키는 과정)
3. Denoising AutoEncoder(DAE)
    - **더 강건한(robust) Feature**를 만들기 위한 AutoEncoder이다.
    - 이를 위해, **Input Data에 약간의 Noise를 추가해서 학습**시킨다.
        - 즉, 어떤 데이터가 Input으로 와도 강건한 모델을 만들겠다는 의미이다.
        
    <참고>
    - Stacked Denoising AutoEncoder(SDAE)는 Stacked AutoEncoder에서 AutoEncoder를 Denoising AutoEncoder로 대체한 모형이다.

In [21]:
# 하이퍼파라미터 지정
BATCH_SIZE = 32  # 미니배치 1개 단위에 대해 데이터가 32개로 구성되어 있는 것을 의미
EPOCHS = 10      # 전체 데이터 셋을 10번 반복해 학습한다는 것을 의미 (즉, 전체 데이터를 이용해 학습을 진행한 횟수)

# FashionMNIST 데이터 다운로드
train_dataset = datasets.FashionMNIST(root = '../data/FashionMNIST',             # 데이터 셋이 저장될 장소를 지정
                                      train = True,                       # 학습용 데이터 셋으로 지정 
                                      download = True,                    # 인터넷 상에서 다운로드해서 이용할 것인지 여부
                                      transform = transforms.ToTensor())  # 데이터 셋을 tensor 형태로 변경
test_dataset = datasets.FashionMNIST(root = '../data/FashionMNIST',
                                     train = False,
                                     download = True,
                                     transform = transforms.ToTensor())

# 다운로드한 FashionMNIST 데이터 셋을 미니배치 단위로 분리
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE, # 미니배치 1개 단위를 구성하는 데이터의 개수
                                           shuffle = True) # 데이터의 순서를 섞고자 할 때 이용(즉, 잘못된 방향으로 학습하는 것을 방지)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [22]:
# 데이터 확인하기 (1)
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [23]:
### 아래의 코드를 실행하면 Kernel이 꺼진다..왜 이러지?
# pltsize = 1
# plt.figure(figsize = (10 * pltsize, pltsize))
# for i in range(10):
#     plt.subplot(1, 10, i + 1)
#     plt.axis('off')
#     plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
#     plt.title('Class: ' + str(y_train[i].item()))

In [24]:
# AutoEncoder(AE) 모델 설계하기
# PyTorch 모듈 내에 딥러닝 모델 관련 기본 함수를 포함하고 있는 nn.Module 클래스를 상속받는 AE 클래스를 정의
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__() # nn.Module 내에 있는 메서드를 상속받아 사용
        
        # 인코더(encoder) 정의 >> nn.Sequential()을 통해 인코더 단위를 한 번에 정의
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 512),  # Input image data
                                     nn.ReLU(),                # 첫 번째 Layer의 Output에 대해 ReLU 함수 적용해, 두 번째 Layer의 Input으로 전달
                                     nn.Linear(512, 256),      # 두 번째 Layer의 Input 크기는 512, Output 크기는 256
                                     nn.ReLU(),                # 두 번째 Layer의 Output에 대해 ReLU 함수 적용해, 두 번째 Layer의 Input으로 전달
                                     nn.Linear(256, 32))       # 두 번째 Layer의 Input 크기는 256, Output 크기는 32
        
        # 디코더(decoder) 정의 >> nn.Sequential()을 통해 인코더 단위를 한 번에 정의
        # 인코더(encoder)와 반대 방향으로 진행한다고 생각하면 쉬움(즉, 원복시키는 방향으로)
        self.decoder = nn.Sequential(nn.Linear(32, 256),       # Latent Variable Vector를 Input으로 이용
                                     nn.ReLU(),
                                     nn.Linear(256, 512),
                                     nn.ReLU(),
                                     nn.Linear(512, 28 * 28))
    
    # Forward Propagation(순전파) 정의
    # 즉, 설계한 AutoEncoder의 인코더와 디코더에 데이터를 입력했을 때, Output을 계산하기까지의 과정을 나열한 것을 의미
    def forward(self, x):
        encoded = self.encoder(x)        # Image 데이터를 사전에 정의한 인코더의 Input으로 이용하여 Latent Variable Vector를 생성
        decoded = self.decoder(encoded)  # Latent Variable Vector 값이 저장된 encoded를 디코더의 Input으로 이용
        return encoded, decoded

In [25]:
# Optimizer, Objective Function 설정하기
model = AE().to(DEVICE) # 'DEVICE' 장비를 이용하여 AE 모델을 완성하기 위해, 앞서 정의한 AE 모델을 기존에 선정한 'DEVICE'에 할당
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # Adam Optimizer를 사용하고, Learning Rate은 0.001로 설정
criterion = nn.MSELoss() # MSE를 모델 평가 지표로 사용

print(model)

AE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=784, bias=True)
  )
)


In [26]:
# AE 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 (AE 모델을 학습)
def train(model, train_loader, optimizer, log_interval):
    model.train() # 기존에 정의한 AE 모델을 '학습' 상태로 지정
    # train_loader 내에 미니배치 단위로 저장된 데이터를 순서대로 이용하여 AE 모델을 학습
    for batch_idx, (image, _) in enumerate(train_loader): # 입력 데이터를 target으로 학습
        
        # 기존에 정의한 AutoEncoder의 Input은 28 * 28 크기의 1차원 Layer이므로, 2차원 image 데이터를 1차원 데이터로 재구성해서 할당
        image = image.view(-1, 28 * 28).to(DEVICE)   # 미니배치 내에 있는 image 데이터를 기존에 정의한 장비에 할당
        target = image.view(-1, 28 * 28).to(DEVICE)  # 미니배치 내에 있는 image 데이터를 AE 모델의 Output과 비교하기 위해 기존에 정의한 장비에 할당
        optimizer.zero_grad() # opimizer의 Gradient를 초기화
        
        encoded, decoded = model(image)    # 장비에 할당한 image 데이터를 AE 모델의 Input으로 이용
        loss = criterion(decoded, target)
        
        loss.backward()   # Loss 값을 계산한 결과를 바탕으로 역전파를 통해 계산된 Gradient 값을 각 파라미터에 할당
        optimizer.step()  # 각 파라미터에 할당된 Gradient 값을 이용하여 파라미터 값을 업데이트
        
        # 아래의 코드는 위 함수가 실행되는 과정을 모니터링하기 위한 코드
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), loss.item()))

In [27]:
# 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 (학습의 진행 과정을 모니터링)
def evaluate(model, test_loader):
    model.eval()      # 학습 과정 또는 학습이 완료된 AE 모델을 학습 상태가 아닌 '평가' 상태로 지정
    test_loss = 0     # 기존에 정의한 test_loader 내의 데이터를 이용하여 Loss 값을 계산하기 위해 '0'으로 임시 설정
    real_image = []   # 학습 과정에서 AutoEncoder에 이용되는 실제 image 데이터 할당용
    gen_image = []    # 학습 과정에서 AutoEncoder를 통해 생성되는 image 데이터 할당용
    
    # AE 모델을 평가하는 단계에서는 Gradient를 통해 파라미터 값이 업데이트 되는 현상을 방지하기 위해,
    # torch.no_grad() 메서드를 통해 Gradient의 흐름을 억제
    with torch.no_grad():
        for image, _ in test_loader:
            # 기존에 정의한 AutoEncoder의 Input은 28 * 28 크기의 1차원 Layer이므로, 2차원 image 데이터를 1차원 데이터로 재구성해서 할당
            image = image.view(-1, 28 * 28).to(DEVICE)   # 미니배치 내에 있는 image 데이터를 기존에 정의한 장비에 할당
            target = image.view(-1, 28 * 28).to(DEVICE)  # 미니배치 내에 있는 image 데이터를 AE 모델의 Output과 비교하기 위해 기존에 정의한 장비에 할
            
            encoded, decoded = model(image) # 장비에 할당한 image 데이터를 AE 모델의 Input으로 사용
            test_loss += criterion(decoded, image).item() # test_loss 값 업데이트
            
            real_image.append(image.to('cpu'))   # 실제 이미지로 할당된 이미지를 real_image 리스트에 추가
            gen_image.append(decoded.to('cpu'))  # AutoEncoder 모델을 통해 생성된 이미지를 gen_image 리스트에 추가
    
    test_loss /= len(test_loader.dataset) # 현재까지 계산된 test_loss 값을 미니배치 개수만큼 나눠서 평균 Loss 값을 계산
    
    return test_loss, real_image, gen_image

In [28]:
# AutoEncoder 학습을 실행하며 Test set의 Reconstruction Error 확인하기
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, real_image, gen_image = evaluate(model, test_loader)
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}'.format(epoch, test_loss))
    
#     # 실제 이미지와 생성된 이미지를 비교해서 학습의 진행도를 확인 >> 아래의 코드를 실행하면 Kernel이 꺼진다..왜 이러지?
#     f, a = plt.subplots(2, 10, figsize = (10, 4))
#     # real_image 출력(1행 1열 ~ 10열)
#     for i in range(10):
#         img = np.reshape(real_image[0][i], (28, 28))
#         a[0][i].imshow(img, cmap = 'gray_r')
#         a[0][i].set_xticks(())
#         a[0][i].set_yticks(())
#     # gen_image 출력(2행 1열 ~ 10열)
#     for i in range(10):
#         img = np.reshape(gen_image[0][i], (28, 28))
#         a[1][i].imshow(img, cmap = 'gray_r')
#         a[1][i].set_xticks(())
#         a[1][i].set_yticks(())
        
#     plt.show()

Train Epoch: 1 [0/60000 (0%)]	Train Loss: 0.197536
Train Epoch: 1 [6400/60000 (11%)]	Train Loss: 0.027059
Train Epoch: 1 [12800/60000 (21%)]	Train Loss: 0.025013
Train Epoch: 1 [19200/60000 (32%)]	Train Loss: 0.024060
Train Epoch: 1 [25600/60000 (43%)]	Train Loss: 0.017537
Train Epoch: 1 [32000/60000 (53%)]	Train Loss: 0.015437
Train Epoch: 1 [38400/60000 (64%)]	Train Loss: 0.016734
Train Epoch: 1 [44800/60000 (75%)]	Train Loss: 0.017141
Train Epoch: 1 [51200/60000 (85%)]	Train Loss: 0.016448
Train Epoch: 1 [57600/60000 (96%)]	Train Loss: 0.015457

[EPOCH: 1], 	Test Loss: 0.0005
Train Epoch: 2 [0/60000 (0%)]	Train Loss: 0.016329
Train Epoch: 2 [6400/60000 (11%)]	Train Loss: 0.014880
Train Epoch: 2 [12800/60000 (21%)]	Train Loss: 0.012787
Train Epoch: 2 [19200/60000 (32%)]	Train Loss: 0.013817
Train Epoch: 2 [25600/60000 (43%)]	Train Loss: 0.014479
Train Epoch: 2 [32000/60000 (53%)]	Train Loss: 0.015783
Train Epoch: 2 [38400/60000 (64%)]	Train Loss: 0.013005
Train Epoch: 2 [44800/60000 